In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import vis.classification_vis as cvis
from Evals import *
from Data_Provider import *
import util.classification_utils as util
import util.data_utils as dutil
import util.label_utils as lutil


import numpy as np
from scipy import interp
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import os

from itertools import product
from multiprocessing import Pool


In [28]:
# Run classifier with cross-validation
def calc_results_and_save(x, y, configs, shuff):
    cv = StratifiedKFold(n_splits=10, shuffle=shuff, random_state=1)
    #dataframe for saving results
    results = pd.DataFrame(columns=('Number Estimators','Max Depth','Max Features','AVG PR'))#,'AUC ROC'))
    #do random search on parameters
    est = np.random.choice(np.arange(130)[5:],60)
    max_d = np.random.choice(np.arange(60)[1:],60)
    max_f = np.random.choice(np.arange(min(x.shape))[1:],60)

    #Search for best hyperpara combo with cross validation
    for idx,(c,d,f) in enumerate(zip(est,max_d,max_f)):
        classifier = RandomForestClassifier(n_estimators=c, max_depth=d, max_features=f,random_state=0) #init classifier
        auc_pr = util.get_auc_score(classifier, cv, x, y, go_after_pr=True)
        #auc_roc =util.get_auc_score(classifier, cv, x, y, go_after_pr=False)
        results.loc[idx] = [c,d,f,auc_pr]
        print('Number Estimators= %d, Max Depth = %d, Max Feat = %d, AUC PR %.3f' % (c,d,f,auc_pr))
    if shuff:
        dutil.save_results(results,configs,'RF') # save results for later use
    else:
        dutil.save_results(results,configs,'RF_NoShuffle') # save results for later use      
    return results


def vis_results(x,y, x_ev, y_ev, hypers):
    best_n, best_d, best_f = hypers
    ###using hyperpara found, evaluate and get pretty plots
    #get f1 scores on whole training set
    cv = StratifiedKFold(n_splits=10, shuffle=False, random_state=1)
    classifier = RandomForestClassifier(n_estimators=int(best_n), max_depth=int(best_d), max_features=int(best_f), random_state=0)
    best_thr = util.get_optimal_threshold(classifier, cv, x, y, go_after_pr=True) # get threshold using cv (on whole dataset)
    y_pred,y_pred_ev = util.fit_predict(classifier, best_thr, x, y, x_ev, y_ev) # using that threshold, get predictions and f1 score
    f1_tr=get_f1(y_pred,y) # calculate f1 scores for prediction on train set
    f1_ev=get_f1(y_pred_ev,y_ev)
    prec_tr,recall_tr = get_precision_recall(y_pred,y)
    prec_ev,recall_ev = get_precision_recall(y_pred_ev,y_ev)

    print('Training, Precision: {0:.3f}, Recall: {1:.3f}, F1: {2:.3f}'.format(prec_tr, recall_tr,f1_tr))
    print('Eval, Precision: {0:.3f}, Recall: {1:.3f}, F1: {2:.3f}'.format(prec_ev, recall_ev,f1_ev))
    #draw pretty plots
    cvis.conf_mat(y_pred,y)
    cvis.conf_mat(y_pred_ev,y_ev)


    cvis.plot_roc(x,y,classifier, 'Random Forest, n_estimators = {}, max_depth={}, max_features = {}'.format(best_n,best_d,best_f))
    cvis.plot_roc(x_ev,y_ev,classifier, 'Random Forest, n_estimators = {}, max_depth={}, max_features = {}'.format(best_n,best_d,best_f))
    cvis.plot_pr_curve(x,y,classifier,'Random Forest, n_estimators = {}, max_depth={}, max_features = {}'.format(best_n,best_d,best_f))
    cvis.plot_pr_curve(x_ev,y_ev,classifier,'Random Forest, n_estimators = {}, max_depth={}, max_features = {}'.format(best_n,best_d,best_f))

    
def do_all(file, cut, shuffled=False, random = False):
# def do_all(data, cut, shuffled=False, random = True):
    provider = DataProvider()
    configs = dutil.generate_configs_from_file(file, cut)
    print(configs)
    x,y,x_ev,y_ev = provider.get_data(configs)
    #x,y,x_ev,y_ev = data
    #if random:
    if False:
        print('yes, random')
        np.random.seed()
        y = randomize_labels(y)
        y_ev = randomize_labels(y_ev)
    res = calc_results_and_save(x,y,configs,shuffled)
    #res = calc_results_and_save(x,y,shuffled)
    ####
#     pos = res['AVG PR'].idxmax()
#     best_row=list(res.loc[pos].values) # get the row with highest ev score
#     best_row.append(y)
#     best_row.append(y_ev)
#     return best_row
    ####
    #hypers = util.get_best_hyperparas(res,'AVG PR')
    #vis_results(x,y, x_ev, y_ev,hypers)
    
def randomize_labels(y):
    ones = int(np.sum(y))
    #where?
    fill_ones = np.random.choice(len(y),ones, replace=False)
    ret = np.empty(len(y))
    ret[:]=0
    ret[fill_ones]=1
    return ret
    
    

In [35]:
files = [f for f in os.listdir('/home/emil/OpenMindv2/data/several_days') if f.endswith('hdf')][3]
cuts = [.1,.2,.3]
shuffled =[True,False]
all_elements = [[files],cuts, shuffled]
#all_elements = [[bla],cuts]

file_cut_combos = []
for allel in product(*all_elements):
    file_cut_combos+=[allel]

In [36]:
files

'wsize_100_sliding_25_s_sample_0_e_sample_150000_s_sample_ev_150000_e_sample_ev_190000_expvar_90.hdf'

In [ ]:
pool = mp.Pool(6)
yass = pool.starmap(do_all,file_cut_combos)


{'wsize': 100, 'sliding': 25, 's_sample': 0, 'e_sample': 150000, 's_sample_ev': 150000, 'e_sample_ev': 190000, 'expvar': 90, 'cutoff': 0.2}
{'wsize': 100, 'sliding': 25, 's_sample': 0, 'e_sample': 150000, 's_sample_ev': 150000, 'e_sample_ev': 190000, 'expvar': 90, 'cutoff': 0.2}
{'wsize': 100, 'sliding': 25, 's_sample': 0, 'e_sample': 150000, 's_sample_ev': 150000, 'e_sample_ev': 190000, 'expvar': 90, 'cutoff': 0.3}
{'wsize': 100, 'sliding': 25, 's_sample': 0, 'e_sample': 150000, 's_sample_ev': 150000, 'e_sample_ev': 190000, 'expvar': 90, 'cutoff': 0.1}
{'wsize': 100, 'sliding': 25, 's_sample': 0, 'e_sample': 150000, 's_sample_ev': 150000, 'e_sample_ev': 190000, 'expvar': 90, 'cutoff': 0.3}
{'wsize': 100, 'sliding': 25, 's_sample': 0, 'e_sample': 150000, 's_sample_ev': 150000, 'e_sample_ev': 190000, 'expvar': 90, 'cutoff': 0.1}
/home/emil/OpenMindv2/data/several_days/wsize_100_sliding_25_s_sample_0_e_sample_150000_s_sample_ev_150000_e_sample_ev_190000_expvar_90.hdf
/home/emil/OpenMindv

Number Estimators= 65, Max Depth = 10, Max Feat = 21, AUC PR 0.624
Number Estimators= 79, Max Depth = 37, Max Feat = 19, AUC PR 0.325
Number Estimators= 79, Max Depth = 37, Max Feat = 19, AUC PR 0.188
Number Estimators= 79, Max Depth = 37, Max Feat = 19, AUC PR 0.828
Number Estimators= 99, Max Depth = 50, Max Feat = 12, AUC PR 0.100
Number Estimators= 99, Max Depth = 50, Max Feat = 12, AUC PR 0.679
Number Estimators= 125, Max Depth = 44, Max Feat = 11, AUC PR 0.771
Number Estimators= 125, Max Depth = 44, Max Feat = 11, AUC PR 0.328
Number Estimators= 125, Max Depth = 44, Max Feat = 11, AUC PR 0.842
Number Estimators= 22, Max Depth = 10, Max Feat = 11, AUC PR 0.355
Number Estimators= 22, Max Depth = 10, Max Feat = 11, AUC PR 0.721
Number Estimators= 7, Max Depth = 50, Max Feat = 13, AUC PR 0.347
Number Estimators= 7, Max Depth = 50, Max Feat = 13, AUC PR 0.612
Number Estimators= 22, Max Depth = 10, Max Feat = 11, AUC PR 0.791
Number Estimators= 125, Max Depth = 44, Max Feat = 11, AUC PR